In [175]:
# notebook for testing 

from rdflib import Graph, URIRef, Literal, Namespace, RDF, RDFS

In [156]:
def get_points(graph):
    
    points = dict()
    
    points['observables'] = []
    points['actuatables'] = []
    points['just_quantifiables'] = []

    for subject, _, obj in graph.triples((None, RDF.type, S223.QuantifiableObservableProperty)):
        points['observables'].append(subject)

    for subject, _, obj in graph.triples((None, RDF.type, S223.QuantifiableActuatableProperty)):
        points['actuatables'].append(subject)

    for subject, _, obj in graph.triples((None, RDF.type, S223.QuantifiableProperty)):
        points['just_quantifiables'].append(subject)

    return points

In [157]:
def increase_by_one():
    number = 0
    while True: 
        yield number
        number += 1

In [169]:
def add_bacnet_ref(model, graph, id_counter, point, bacnet_type):
    
    # graph and model separate just for debugging

    # could do this based on label alternatively to uri 
    point_name = point.rsplit('/')[-1]

    ref_name = point_name + '_bacnet'
    graph.add((BLDG[ref_name], RDF.type, REF.BACnetReference))
    graph.add((BLDG[ref_name], BACNET['object-type'], Literal(bacnet_type)))
    
    id_num = next(id_counter)
    graph.add((BLDG[ref_name], BACNET['object-identifier'], Literal(f"{bacnet_type},{id_num}")))
    graph.add((BLDG[ref_name], BACNET['object-name'], Literal(point_name)))
    graph.add((BLDG[ref_name], BACNET['objectOf'], BLDG['boptest-proxy']))

    unit = model.value(point, QUDT.hasUnit)
    unit_name = unit.rsplit('/')[-1]

    graph.add((BLDG[ref_name], BACNET['units'],Literal(unit_name)))

    graph.add((point, REF.hasExternalReference, BLDG[ref_name]))

In [170]:
def add_bacnet_device(graph, device_id):
    graph.add((BLDG['boptest-proxy'], RDF.type, BACNET.BACnetDevice))
    graph.add((BLDG['boptest-proxy'], BACNET['device-instance'], Literal(599)))

In [171]:
graph = Graph()

BLDG = Namespace("urn:bldg/")
graph.bind("bldg", BLDG)

REF = Namespace("https://brickschema.org/schema/Brick/ref#")
graph.bind("ref", REF)

BACNET = Namespace("http://data.ashrae.org/bacnet/2020#")
graph.bind("bacnet", BACNET)

S223 = Namespace("http://data.ashrae.org/standard223#")
graph.bind("s223", S223)

QUDT = Namespace("http://qudt.org/schema/qudt/")
graph.bind("qudt", QUDT)

In [172]:
model = Graph()
model.parse("multizone_office_simple_air_223p.ttl", format = "ttl")

<Graph identifier=Nf3a63fa5d55143e090c326a5f833b7ae (<class 'rdflib.graph.Graph'>)>

In [173]:
points = get_points(model)

id_counter = increase_by_one()

# have to add boptest proxy
add_bacnet_device(graph, 599)

for point in points['actuatables']:
    add_bacnet_ref(model, graph, id_counter, point, "analog-output")

for point in points['observables']:
    add_bacnet_ref(model, graph, id_counter, point, "analog-input")

for point in points['just_quantifiables']:
    add_bacnet_ref(model, graph, id_counter, point, "analog-value")

In [174]:
graph.print()

@prefix bacnet: <http://data.ashrae.org/bacnet/2020#> .
@prefix bldg: <urn:bldg/> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

bldg:LowerSetp_cor ref:hasExternalReference bldg:LowerSetp_cor_bacnet .

bldg:LowerSetp_eas ref:hasExternalReference bldg:LowerSetp_eas_bacnet .

bldg:LowerSetp_nor ref:hasExternalReference bldg:LowerSetp_nor_bacnet .

bldg:LowerSetp_sou ref:hasExternalReference bldg:LowerSetp_sou_bacnet .

bldg:LowerSetp_wes ref:hasExternalReference bldg:LowerSetp_wes_bacnet .

bldg:UpperSetp_cor ref:hasExternalReference bldg:UpperSetp_cor_bacnet .

bldg:UpperSetp_eas ref:hasExternalReference bldg:UpperSetp_eas_bacnet .

bldg:UpperSetp_nor ref:hasExternalReference bldg:UpperSetp_nor_bacnet .

bldg:UpperSetp_sou ref:hasExternalReference bldg:UpperSetp_sou_bacnet .

bldg:UpperSetp_wes ref:hasExternalReference bldg:UpperSetp_wes_bacnet .

bldg:hvac_oveZonSupCor_TZonCooSet_u ref:hasExternalReference bldg:hvac_oveZ